In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils.diff import diff

In [2]:
# 64.  rd
# 季度频率。当研发费用占总资产的比例增加大于5%时，指标变量为1。
df1 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计'], low_memory=False)
df2 = pd.read_csv('../../data/csmar/季_利润表.csv', usecols=['证券代码', '会计期间', '研发费用', '营业收入', '查询成功'])
df2['研发费用'][df2['查询成功'].astype('bool')] = df2['研发费用'][df2['查询成功'].astype('bool')].fillna(0)
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3 = diff(df3, col=['研发费用', '营业收入'])
df3['rd1'] = df3['研发费用']/df3['资产总计'].replace(0, np.nan)
df3['rd2'] = (df3['rd1'] - df3.groupby('证券代码').shift(1)['rd1'])/df3.groupby('证券代码').shift(1)['rd1']
df3['rd'] = np.where(df3['rd2'] > 0.05, 1, 0)
df3['rd'][df3['rd2'].isna()] = np.nan
df3['rd'][(df3.groupby('证券代码').shift(1)['rd1'] == 0) & (df3['rd1'] == 0)] = 0
df3['rd'][(df3.groupby('证券代码').shift(1)['rd1'] == 0) & (df3['rd1'] > 0)] = 1

# 65.  rd_mve
# 季度频率。研发费用除以季度末市值。
df4 = pd.read_csv('../../data/csmar/季_相对价值指标.csv', usecols=['证券代码', '会计期间', '市值A'])
df5 = pd.merge(df3, df4, on=['证券代码', '会计期间'], how='left')
df5['rd_mve'] = df5['研发费用']/df5['市值A'].replace(0, np.nan)

# 66.  rd_sale
# 季度频率。研发费用除以季度销售额。
df5['rd_sale'] = df5['研发费用']/df5['营业收入'].replace(0, np.nan)

df6 = df5[['证券代码', '会计期间', 'rd', 'rd_mve', 'rd_sale']]
df6.to_csv('../../output/csmar/季_64_65_66.csv', index=False, encoding='utf-8-sig')
df6

,证券代码,会计期间,rd,rd_mve,rd_sale
0,1,1990-03-31,NaN,NaN,NaN
1,1,1990-06-30,NaN,NaN,NaN
2,1,1990-09-30,NaN,NaN,NaN
3,1,1990-12-31,NaN,NaN,NaN
4,1,1991-03-31,NaN,NaN,NaN
...,...,...,...,...,...
728835,900957,2022-03-31,0.0,0.0,0.0
728836,900957,2022-06-30,0.0,0.0,0.0
728837,900957,2022-09-30,0.0,0.0,0.0
728838,900957,2022-12-31,0.0,0.0,0.0
